In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Librerías para trabajar con fechas
# -----------------------------------------------------------------------
from datetime import datetime

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
#pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [2]:
df = pd.read_csv('../data/bookings.csv', index_col=0)

In [3]:
df

,index,hotel,is_canceled,arrival_date_year_x,arrival_date_month_x,arrival_date_week_number,arrival_date_day_of_month_x,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests,reservation_status,change_room,adr,lead_time,stays_in_weekend_nights,stays_in_week_nights,total_nights,arrival_date_day_of_month_y,arrival_date_month_y,arrival_date_year_y,num_month,fecha
0,0,Resort Hotel,No Cancelado,2015.0,July,27.0,1.0,2,0,0,BB,Portugal,Unknown,Direct,First time,0,0,C,C,3,Unknown,Unknown,0,Transient,0,0,Check-Out,equal,0.000,342,0,0,0,1.0,July,2015.0,7.0,1-7-2015
1,1,Resort Hotel,No Cancelado,2015.0,July,27.0,1.0,2,0,0,BB,Unknown,Unknown,Direct,First time,0,0,Unknown,C,4,Unknown,Unknown,0,Transient,0,0,Check-Out,change,0.000,737,0,0,0,1.0,July,2015.0,7.0,1-7-2015
2,2,Resort Hotel,No Cancelado,2015.0,July,27.0,1.0,1,0,0,BB,United Kingdom,Unknown,Direct,First time,0,0,A,C,0,Unknown,Unknown,0,Transient,0,0,Check-Out,change,75.000,7,0,1,1,1.0,July,2015.0,7.0,1-7-2015
3,3,Resort Hotel,No Cancelado,2015.0,July,27.0,1.0,1,0,0,BB,United Kingdom,Corporate,Corporate,First time,0,0,A,A,0,304.0,Unknown,0,Transient,0,0,Check-Out,equal,75.000,13,0,1,1,1.0,July,2015.0,7.0,1-7-2015
4,4,Resort Hotel,No Cancelado,2015.0,July,NaN,1.0,2,0,0,BB,Unknown,Online TA,TA/TO,First time,0,0,A,A,0,240.0,Unknown,0,Transient,0,1,Check-Out,equal,98.000,14,0,2,2,1.0,July,2015.0,7.0,1-7-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119831,182872,Unknown,Unknown,NaN,Unknown,NaN,NaN,2,0,0,Unknown,Unknown,Unknown,Unknown,First time,0,0,Unknown,Unknown,0,Unknown,Unknown,0,Unknown,0,0,Unknown,change,93.054,224,19,19,38,NaN,NaN,NaN,NaN,NaN
119832,182873,Unknown,Unknown,NaN,Unknown,NaN,NaN,2,0,0,Unknown,Unknown,Unknown,Unknown,First time,0,0,Unknown,Unknown,0,Unknown,Unknown,0,Unknown,0,0,Unknown,change,62.622,390,14,19,33,NaN,NaN,NaN,NaN,NaN
119833,182874,Unknown,Unknown,NaN,Unknown,NaN,NaN,2,0,0,Unknown,Unknown,Unknown,Unknown,First time,0,0,Unknown,Unknown,0,Unknown,Unknown,0,Unknown,0,0,Unknown,change,91.604,230,11,20,31,NaN,NaN,NaN,NaN,NaN
119834,182875,Unknown,Unknown,NaN,Unknown,NaN,NaN,2,0,0,Unknown,Unknown,Unknown,Unknown,First time,0,0,Unknown,Unknown,0,Unknown,Unknown,0,Unknown,0,0,Unknown,change,135.770,304,16,15,31,NaN,NaN,NaN,NaN,NaN


In [4]:
columnas = ['hotel', 'arrival_date_year_x',
       'arrival_date_month_x', 'arrival_date_week_number',
       'arrival_date_day_of_month_x', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces',
       'total_of_special_requests', 'reservation_status', 'change_room', 'adr',
       'lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
       'total_nights', 'fecha']

In [5]:
def calculo_tasa (dataframe, id, columna, columna_tasa, valor_0, valor_1, ruta_csv):
    df = dataframe.groupby([columna, columna_tasa]).count()
    df.reset_index(inplace=True)
    
    df_0 = df[df[columna_tasa] == valor_0][[columna, id]]
    df_1 = df[df[columna_tasa] == valor_1][[columna, id]]

    # Cambiar nombre columnas calculadas
    dict_0 = {f'{id}' : 'num_cancelados'}  
    dict_1 = {f'{id}' : 'num_no_cancelados'}

    df_0.rename(columns=dict_0, inplace=True)
    df_1.rename(columns=dict_1, inplace=True)

    df_final = df_0.merge(df_1, how='inner', on=columna)
    df_final['tasa'] = round((df_final['num_cancelados']/(df_final['num_no_cancelados']+df_final['num_cancelados'])), 2)

    df_final.to_csv(ruta_csv)
    return df_final

In [6]:
def reemplaza_punto_cero(cadena):
    try:
        return cadena.replace('.0', '')
    except:
        return cadena
    
df['agent'] = df['agent'].apply(reemplaza_punto_cero)
df['company'] = df['company'].apply(reemplaza_punto_cero)


array(['Unknown', '110', '270', '240', '154', '144', '307', '268', '59',
       '204', '312', '94', '174', '223', '317', '281', '274', '53', '12',
       '47', '324', '342', '373', '286', '383', '86', '113', '397', '392',
       '331', '367', '20', '416', '51', '395', '102', '34', '405', '394',
       '461', '178', '421', '308', '135', '504', '269', '498', '390',
       '195', '513', '203', '263', '477', '521', '169', '355', '531',
       '62', '120', '82', '81', '116', '530', '103', '9', '39', '92',
       '31', '118', '356', '457', '165', '291', '292', '290', '43', '325',
       '192', '108', '200', '224', '388', '287', '297', '490', '482',
       '207', '437', '329', '28', '77', '42', '338', '83', '72', '146',
       '380', '323', '511', '88', '371', '16', '337', '403', '399', '137',
       '343', '346', '347', '289', '353', '54', '159', '358', '361', '99',
       '366', '372', '365', '14', '377', '379', '378', '330', '22', '364',
       '360', '401', '232', '255', '384', '167', '21

In [7]:
# calculo_tasa (dataframe, id, columna, columna_tasa, valor_0, valor_1, ruta_csv)
for col in columnas:
    calculo_tasa (df, 'index', col, 'is_canceled', 'Cancelado', 'No Cancelado', f'../data/data_tasa/tasa_{col}.csv')